# Imports and Constants

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
import random
import os

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *

C:\Users\nicol\miniconda3\lib\site-packages\nilearn\datasets\__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [4]:
from_who = "our"   # 2 possibilities : "mohameds" or "our"
maps_folder="brain_maps/"+from_who+"_maps"
masks_folder="masks/"+from_who+"_masks"

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
class_labels = [""] * (2 * length_one_modality)
for i in range(length_one_modality * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length_one_modality])
labels["aud"] = np.array(class_labels[length_one_modality:])
labels_same = labels["vis"]

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                           (["vis"], ["PT_L", "PT_R"]),
                           (["aud"], ["V5_L", "V5_R"]),
                           (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                            (["vis", "aud"], ["PT_L", "PT_R"])]

scaler = sklearn.preprocessing.StandardScaler()
classifier = sklearn.svm.SVC(C = 1, kernel = 'linear', random_state=SEED)  # other parameters are default parameters
pipeline = Pipeline([('std_scaler', scaler), ('svm', classifier)])
decoder = Decoder(n_perm=n_individual_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
_ = decoder.return_and_reset_predictions(classes)

# Loading data

In [5]:
maps_masked, masks_exist = load_full_data(subjects_ids, length_one_modality, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))
decoder.set_masks_exist(masks_exist)

# Within-modality decoding

In [ ]:
confusion_matrixes = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
#within_modality_group_results = average_dicos(within_cv_scores)

# Cross-modal decoding

In [9]:
cross_cv_scores = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
cross_modality_group_results = average_dicos(cross_cv_scores)

# Bootstrapping to assess group-level significance

In [7]:
n_single_perm = 100
n_bootstrap = 100_000

In [ ]:
within_scores_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same,within_modal_tasks_regions,maps_masked,n_subjects, within_modality=True)
bootstrapped_distribution_within = compute_bootstrap_distribution(n_bootstrap, n_subjects, within_scores_100_perm, n_single_perm)

In [ ]:
cross_scores_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same,cross_modal_tasks_regions,maps_masked,n_subjects, within_modality=False)
bootstrapped_distribution_cross = compute_bootstrap_distribution(n_bootstrap, n_subjects, cross_scores_100_perm, n_single_perm)

# Saving results

In [ ]:
params = classifier.get_params()
out_directory = "out/"+maps_folder+"_"+masks_folder+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"
create_directory(out_directory)

In [ ]:
save_dicts(out_directory+"confusion_matrixes.csv", confusion_matrixes, list(confusion_matrixes[0].keys()), subjects_ids)
compute_metric(out_directory, subjects_ids, {"name": "accuracy", "function" : accuracy})
compute_metric(out_directory, subjects_ids, {"name": "recall", "function" : recall})
compute_metric(out_directory, subjects_ids, {"name": "precision", "function" : precision})

In [ ]:
#save_dicts(out_directory+"within_cv_scores.csv", within_cv_scores, list(within_cv_scores[0].keys()), subjects_ids)
#save_dicts(out_directory+"group_scores_within.csv", [within_modality_group_results], list(within_modality_group_results.keys()), [0])

In [ ]:
save_dicts(out_directory+"cross_cv_scores.csv", cross_cv_scores, list(cross_cv_scores[0].keys()), subjects_ids)
save_dicts(out_directory+"group_scores_cross.csv", [cross_modality_group_results], list(cross_modality_group_results.keys()), [0])

In [12]:
save_dicts(out_directory+"bootstraps_within.csv", [bootstrapped_distribution_within], list(bootstrapped_distribution_within[0].keys()), range(n_bootstrap))

In [ ]:
save_dicts(out_directory+"bootstraps_cross.csv", [bootstrapped_distribution_cross], list(bootstrapped_distribution_cross[0].keys()), range(n_bootstrap))

# Plotting results (from files of saved results)

In [2]:
results_directory = "out/"+maps_folder+"_"+masks_folder+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"     # to be changed depending on where you want to see the results from
cv_df = retrieve_cv_scores(results_directory)
bootstrap_df  = retrieve_bootstrap_scores(results_directory)

In [ ]:
# compute estimated p-values for bootstrap
pvals = compute_p_val_bootstrap(bootstrap_df, cv_df)
save_dicts(out_directory+"estimated_pval_bootstrap.csv", [pvals], list(pvals.keys()), [0])

In [5]:
plt_directory = "plots/"+maps_folder+"_"+masks_folder+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)
plotter.plot_cv_score(cv_df, chance_level = True)
plotter.plot_bootstrap(bootstrap_df, cv_df, pvals, 30)